# Модуль №1
## 1.1 Предобработка данных и выделение значиммых атрибутов
Для начала работы загрузим нужные библиотеки и откроем файл с данными в виде таблицы DataFrame библиотеки pandas

In [178]:
import pandas as pd
import numpy as np

In [179]:
# Парсинг данных
df=pd.read_excel("data/data.xlsx", sep=";")
df.columns=["id", "no", "date", "origin_date", "recipient", "author", "content", "extra"]
df.head()

,id,no,date,origin_date,recipient,author,content,extra
0,2,WSR/1-837/2019,2019-04-09,05-338\n09.04.2019,Уразов Р.Н.,Картошкин С.А. (Министерство просвещения Росси...,Вх - Об участии в совещании по вопросу доработ...,NaN
1,3,WSR/1-835/2019,2019-04-09,И10-14/3137\n09.04.2019,Уразов Р.Н.,Цивилев С.Е. (Администрация Кемеровской области),Вх - О командировании Саликовой К. и Глушко Д....,NaN
2,4,WSR/1-834/2019,2019-04-09,Д2/7121-ИС\n03.04.2019,Иванюк Л.А.,Волков Г.А. (Министерство транспорта Российско...,Вх - Об участии в Чемпионате мира по профессио...,NaN
3,5,WSR/1-833/2019,2019-04-09,15-20/06-911\n08.04.2019,Уразов Р.Н.,Атанов И.В. (Ставропольский государственный аг...,Вх - Об участии в церемонии открытия Вузовског...,NaN
4,6,WSR/1-831/2019,2019-04-09,Исх01/0403\n02.04.2019,Миронова С.В.,Островский А.В. (Губернатор Смоленской области),Вх - Об участии в Чемпионате мира по профессио...,NaN


Исходя из предметной области, для классификации корреспонденции будущей модели не нужны будут такие данные как дата отправления и создания документа, а также его номер в системе. Поэтому оставим только имя получателя письма (будущую целевую переменную), отправителя, краткое содержание, а также приложения к письмам, т.к. из них в будущем необходимо будет извлечь дополнительную информацию

In [180]:
df=df.drop(["id","no","date","origin_date"], axis=1)
# При повторном запуске ячейки без перекомпиляции выдаст ошибку, т.к. удаляемых
# столбцов уже не будет в таблице после первого запуска
df.shape

(4966, 4)

В результате получаем таблицу из 4967 строк и 4 столбца

## 1.2 Разбиение сложных атрибутов
Сложными атрибутами в данном наборе данных являются значения всех полей, кроме целевого, поэтому нам необходимо будет обработать в ходе сессии все. В разделе 1.2 будет обработано поле получателей, в разделе 1.3 поле отправителей, в разделе 1.4 частично будет затронуто поле содержания писем, а в 1.5 - поле приложений к письмам.

Рассмотрим поле получателей

In [181]:
df["recipient"].value_counts()

Глушко Д.Е.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

Поле хранит в себе конкатенированые через спецсимвол "\n" значения. Разделим их и преобразуем имена в более легко интерпретируемый с помощью кода вид (из "Фамилия И.О." в "ФамилияИО") и применим метод CountVectorizer библиотеки Scikit-learn для получения схожей с One-Hot матрицы, в которой столбцы - все варианты адресатов, а значения в строках - 0 (не адресат) и 1 (адресат)

In [182]:
def processRecs(inp):
    inp=inp.replace(' ','')
    inp=inp.replace('.','')
    inp=inp.replace('\n',' ')
    return inp

df_r=df['recipient'].apply(processRecs)
df_r

0                                          УразовРН
1                                          УразовРН
2                                          ИванюкЛА
3                                          УразовРН
4                                        МироноваСВ
5                                          УразовРН
6                                        ТымчиковАЮ
7                                          УразовРН
8                                          УразовРН
9                                     КрайчинскаяСБ
10                                        АнтоновЮВ
11                                         УразовРН
12                                         УразовРН
13                                       ТымчиковАЮ
14                                         ГлушкоДЕ
15                                         ГлушкоДЕ
16                                         УразовРН
17                                         ГлушкоДЕ
18                                         ГлушкоДЕ
19          

In [183]:
from sklearn.feature_extraction.text import CountVectorizer as CV
CVr = CV()
rec_vecs=CVr.fit_transform(df_r)
CVr.get_feature_names()
# Получившиеся признаки

['абдулгалеевзт',
 'абдулганиевфс',
 'абдуллазяновэю',
 'абдуллинам',
 'аблязовка',
 'абязоваюа',
 'агеевшр',
 'акимовпм',
 'алешинав',
 'алимоваа',
 'антипинса',
 'антоновюв',
 'артемоватв',
 'афанасьеваи',
 'афанасьевасн',
 'афанасьевмп',
 'ахметовмг',
 'ахметшинаи',
 'ахметшинас',
 'ахметшинрк',
 'аштаевакд',
 'аюповаих',
 'багровюн',
 'бадретдиновбм',
 'базероэ',
 'баковаюв',
 'балтусоваоа',
 'бариевии',
 'бариевмм',
 'барышеватг',
 'бастрыкинаи',
 'безносикована',
 'богучарскаяас',
 'большаковав',
 'бородинис',
 'бортниковав',
 'булавинви',
 'бургановрт',
 'вазыховна',
 'валеевдр',
 'валитовиа',
 'валиуллинаа',
 'валиуллинрн',
 'васильеваою',
 'вахитоврр',
 'вишняковаоб',
 'волковаи',
 'выборноваа',
 'габдрашитовбр',
 'габдурахмановлр',
 'газизовах',
 'гайдаевми',
 'гайзатуллинрр',
 'гайнутдиновар',
 'галеевэг',
 'галиуллинрф',
 'гараеваар',
 'гараевзф',
 'гариповим',
 'гатаулинтм',
 'гатауллинаэ',
 'гатиятуллинрх',
 'гафаровмр',
 'гафуровир',
 'гаязовзф',
 'гилмановкк',
 'гильмут

In [184]:
encoded_rec=pd.DataFrame(rec_vecs.toarray())
encoded_rec.columns=CVr.get_feature_names()
encoded_rec

,абдулгалеевзт,абдулганиевфс,абдуллазяновэю,абдуллинам,аблязовка,абязоваюа,агеевшр,акимовпм,алешинав,алимоваа,...,шакировра,шафигуллинлн,шаяхметоврк,шигабутдиновак,шойгуск,щугоревна,юсуповиу,юшкосв,ягфароваию,якубовюд
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


На выходе получаем разреженную матрицу из 281 столбца, которую впоследствии можно будет присоединить к таблице с данными

## 1.3 Дополнение недостающими данными
В данных об отправителе указано как лицо, являющееся отправителем, так и организация, которую оно представляет. Разбиение этого поля на два даст больше информации об отправителе, а также впоследствии поможет выявить особо важные лица (будут встречены чаще других)

In [185]:
df['author'].value_counts()

Черноскутова И.А. (Министерство образования и науки Российской Федерации. Департамент государственной политики в сфере подготовки рабочих кадров и ДПО)     93
Картошкин С.А. (Министерство образования и науки Российской Федерации. Департамент государственной политики в сфере подготовки рабочих кадров и ДПО)        41
Потехина И.П. (Министерство образования и науки Российской Федерации)                                                                                       39
Шевелева А.А. (Министерство образования Тульской области)                                                                                                   38
Соляников Ю.В. (Комитет по образованию Правительства Санкт-Петербурга)                                                                                      37
Губайдуллин Э.Ф. (Автономная некоммерческая организация "Центр развития профессиональных компетенций")                                                      34
Шохин А. (Национальный совет при Президенте РФ

Для этого определим две функции: первая возвращает данные внутри скобок, а вторая - данные до скобок, а после - сохраним обработанный обеими функциями столбец в две различных переменных

In [186]:
def getBracketContent(inp):
    pos=inp.find('(')
    if (pos==-1):
        return inp
    return inp[pos+1:len(inp)-1]

def getBeforeBracket(inp):
    pos=inp.find('(')
    if (pos==-1):
        return inp
    if (inp.find('инистерство')==-1):
        return inp[:pos-1]
    return None

In [187]:
authors=df['author'].apply(getBeforeBracket)
authors.value_counts()

Губайдуллин Э.Ф.         49
Соляников Ю.В.           45
Шохин А.                 27
Артемьев И.А.            25
Волощук Л.В.             25
Щелкун Н.И.              23
Антонова О.Г.            22
Шевцова Т.А.             22
Голикова Т.А.            20
Сабитов А.Р.             19
Лейбович А.Н.            19
Кадыров А.Р.             19
Непогода Л.И.            18
Цуров М.Т.               18
Судленков А.А.           17
Жолобова О.В.            17
Скок Т.А.                17
Райдер А.В.              15
Минина В.В.              15
Голодец О.Ю.             15
Ажгиревич А.И.           15
Смирницкая М.В.          15
Магомедов Ш.М.           15
Гудыно А.В.              14
Калугина К.Е.            14
Данилов-Данильян А.В.    14
Аристова В.А.            14
Юрьева С.Н.              14
Силинг А.Л.              14
Ларионова Н.И.           13
                         ..
Максимова Ю.              1
Хрулев В.Е.               1
Федорова М.Н.             1
Свистунова В.П.           1
Анисимов М.Ю.       

In [188]:
orgs=df['author'].apply(getBracketContent)
orgs.value_counts()

Министерство образования и науки Российской Федерации. Департамент государственной политики в сфере подготовки рабочих кадров и ДПО                                                                                                                           147
Министерство образования и науки Российской Федерации                                                                                                                                                                                                          82
Министерство просвещения Российской Федерации                                                                                                                                                                                                                  64
Аппарат Правительства Российской Федерации                                                                                                                                                                                        

Таким образом, мы получили два новых столбца для нашего набора данных

## 1.4 Формирование словарей данных
В данном разделе нам необходимо построить на основе кратких сведений о содержании писем построить словарь, содержащий в себе слова, которые могут быть использованы для предсказания target-переменной в дальнейшем, а также общее количество вхождений слов в документы.

Для начала этого этапа используем на столбце с текстами задействованный ранее класс CountVectorizer, указав в нем в качестве стоп-слов часто используемые предлоги и различные непонятные символы из данных (были вручную скопированы из получившегося после первого прогона данных через векторайзер словаря)

In [199]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ksv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [203]:
content=df['content']
vectorizer=CV(stop_words=stopwords.words("russian"))
vectors=vectorizer.fit_transform(content)
vectorizer.vocabulary_

{'вх': 935,
 'участии': 4945,
 'совещании': 4353,
 'вопросу': 876,
 'доработки': 1304,
 'фэо': 5079,
 'предоставления': 3519,
 'субсидии': 4613,
 'реализацию': 4003,
 'проекта': 3737,
 'билет': 638,
 'будущее': 679,
 'апр': 529,
 '2019': 83,
 'командировании': 1914,
 'саликовой': 4161,
 'глушко': 1038,
 'участия': 4948,
 'заседании': 1494,
 'оргкомитета': 2873,
 'проведения': 3694,
 'фнч': 5048,
 'кемеровской': 1851,
 'области': 2663,
 '22': 107,
 'чемпионате': 5173,
 'мира': 2357,
 'профессиональному': 3822,
 'мастерству': 2220,
 'стандартам': 4548,
 'ворлдскиллс': 880,
 'worldskills': 388,
 'kazan': 344,
 'церемонии': 5131,
 'открытия': 2969,
 'вузовского': 932,
 'чемпионата': 5170,
 '15': 46,
 '14': 39,
 '00': 0,
 'ставрополь': 4540,
 'направлении': 2484,
 'протокола': 3801,
 'ходе': 5099,
 'реализации': 4002,
 'национального': 2522,
 'демография': 1180,
 'субъектах': 4620,
 'россии': 4109,
 'согласовании': 4379,
 'дат': 1134,
 'национальном': 2524,
 'проведении': 3692,
 'заседания'

Ниже представлены вхождения слов (столбцы) в описание писем (индекс=строка) в виде разреженной матрицы, где значения ячейки - количество вхождений слова в документ

In [192]:
vecs_df=pd.DataFrame(vectors.toarray())
vecs_df.columns=vectorizer.get_feature_names()
vecs_df

,academy,airlung,amberforum,arena,articskills,atomskills,autodesk,budapest,cad,cd,...,ялтинском,ялтинскому,ямедовой,янао,янв,январе,января,янтарной,ярославль,ярославская
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Получившееся количество слов в словаре - всего 5132, что объясняется схожей тематикой текстов и малым количеством образцов 
## 1.5 Преобразование переадресаций
При переадресации в исходных данных в поле "приложения" указывается лицо, которому было переадресовано письмо. Эту информацию необходимо вычленить. Проблему в данном случае составляет наличие не нужных в рамках нашей задачи приложений, а также нестандартизированность записей в этом поле. Для того, чтобы устранить эту проблему, необходимо провести обработку этого поля.

Создадим метод, определяющий записи о переадресации по наличию в них слов "передан" и "направлено", а также стирающий информацию о дате, и применим его к столбцу с приложениями

In [193]:
def filterExtras(inp):
    if (str(inp).lower().find('направлено')!=-1 or str(inp).lower().find('передан')!=-1):
        # Некоторые записи содержат ключевые слова, но не являются сведениями о переадресации конкретному лицу
        #Следующее условие их отфильтрует:
        if (str(inp.lower()).find('@')==-1 and str(inp.lower()).find('департамент')==-1 and str(inp.lower()).find('бухгалт')==-1 and str(inp.lower()).find('отдел')==-1 and str(inp).lower().find('академ')==-1 and str(inp).lower().find('адресат')==-1 and str(inp.lower()).find('суд')==-1):
            inp=inp.replace('1','')
            inp=inp.replace('2','')
            inp=inp.replace('3','')
            inp=inp.replace('4','')
            inp=inp.replace('5','')
            inp=inp.replace('6','')
            inp=inp.replace('7','')
            inp=inp.replace('8','')
            inp=inp.replace('9','')
            inp=inp.replace('0','')
            inp=inp.replace('- ','')
            inp=inp.replace('-','')
            inp=inp.replace('.. ','')
            inp=inp.replace(' ..','')
            inp=inp.replace('..','')
            inp=inp.replace('\n',' ')
            return inp
    return None

Посмотрим, какие значения остались в этом столбце после частичной обработки:

In [194]:
df['extra']=df['extra'].apply(filterExtras)
df['extra'].value_counts()

Направлено Митькиной А.                                                                                                         10
Оригиналы приложений переданы Смирновой Л.                                                                                       5
Направлено Мироновой С.                                                                                                          4
Документы сшиты. Переданы Гайдаеву М.                                                                                            3
документ передан Смирновой Л.                                                                                                    2
Оригиналы приложений переданы Антонову Ю.                                                                                        2
Договор передан Богучарской А.                                                                                                   2
Оригиналы приложения переданы Орловой О.                                           

## 1.6 Подготовка отчета
Для подготовки отчета добавим полученные в ходе сессии данные в таблицу DataFrame и сохраним ее в формате CSV

In [195]:
data2=pd.concat([df, pd.DataFrame(encoded_rec), pd.DataFrame(authors), pd.DataFrame(orgs), pd.DataFrame(vecs_df)], axis=1)

In [196]:
data2.to_csv("data/data2.csv", sep=",")
data2.shape

(4966, 5417)

Получившийся файл data2.csv является конкатенацией исходных и преобразованных данных. Исходные данные не удаляются из набора на случай потенциальной необходимости. Представленный ниже код также сохраняет аналогичный файл, но уже без необработанных столбцов под названием data_clear.csv. Оба файла составляют содержимое архива Data.zip

In [197]:
data3=pd.concat([pd.DataFrame(encoded_rec), pd.DataFrame(vecs_df)], axis=1)
data3.shape

(4966, 5411)

In [198]:
data3.to_csv("data/data_clear.csv", sep=",")


В качестве отчета по результатам сессии предоставляю данный Jupyter Notebook